Run this notebook in the same virtual environment with superlinked server
to ensure the same version of `superlinked` framework.

In [1]:
!pip freeze | grep superlinked

superlinked==29.1.0
superlinked-server==1.37.0


In [1]:
from pathlib import Path
import sys

# Determine current working directory
cwd = Path.cwd()

# Adapt logic for your specific project structure
if cwd.name == "intelligent-file-search":
    project_dir = cwd
elif cwd.name == "notebooks":
    project_dir = cwd.parent
else:
    # fallback (e.g., if inside a subfolder deeper in notebooks or dataset)
    project_dir = cwd
    while project_dir.name != "intelligent-file-search" and project_dir != project_dir.parent:
        project_dir = project_dir.parent

superlinked_app_dir = project_dir / "superlinked_app"
assert superlinked_app_dir.exists(), (
    f"{superlinked_app_dir} does not exist\n"
    "Are you sure you are in or below the intelligent-file-search directory?"
)

if str(project_dir) not in sys.path:
    sys.path.append(str(project_dir))
    print("project_dir is added to sys.path")
else:
    print("project_dir is already in sys.path")
print(project_dir)
print(sys.path)

project_dir is added to sys.path
/home/biso/development/my_projects/intelligent-file-search
['/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/home/biso/development/my_projects/.venv/lib/python3.12/site-packages', '/home/biso/development/my_projects/intelligent-file-search']


### This is to use the collection_name=filesearch or file_monkesearch

In [2]:
import os
#os.environ["APP_ID"] = "filesearch"
os.environ["APP_ID"] = "file_monkesearch"


In [5]:
print(cwd)

/home/biso/development/my_projects/intelligent-file-search/notebooks


In [4]:
# Patch .env variables to absolute paths
import os
os.chdir("..")  # Move up to /intelligent-file-search

In [6]:

from superlinked import framework as sl

#from superlinked_app.index import file_index, file_schema
from superlinked_app.index import file_index, file_schema
from superlinked_app.query import query
from superlinked_app.api import vector_database

import pandas as pd

Loaded categories: ['css', 'csv', 'epub+zip', 'geo+json', 'gif', 'html', 'java-archive', 'javascript', 'jpeg', 'json', 'markdown', 'mp4', 'msword', 'octet-stream', 'ogg', 'pdf', 'pgp-keys', 'pkcs12', 'plain', 'png', 'svg+xml', 'unknown', 'vnd.a', 'vnd.android.package-archive', 'vnd.exstream-package', 'vnd.groove-tool-template', 'vnd.microsoft.icon', 'vnd.ms-excel', 'vnd.oasis.opendocument.spreadsheet', 'vnd.oasis.opendocument.text', 'vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'vnd.openxmlformats-officedocument.wordprocessingml.document', 'webm', 'x-diff', 'x-matroska', 'x-ms-wmv', 'x-msdos-program', 'x-object', 'x-python', 'x-sh', 'xml', 'zip']
10:56:04 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/paraphrase-MiniLM-L3-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

10:56:09 superlinked.framework.common.space.embedding.model_based.embedding_engine_manager INFO   Consider caching model dimension.
10:56:10 superlinked.framework.common.space.embedding.model_based.embedding_engine_manager INFO   Consider caching model dimension.
10:56:10 superlinked.framework.common.space.embedding.model_based.embedding_engine_manager INFO   Consider caching model dimension.
10:56:10 superlinked.framework.dsl.index.index INFO   initialized index


In [8]:
source = sl.InteractiveSource(file_schema)
executor = sl.InteractiveExecutor(
    sources=[source],
    indices=[file_index],
    vector_database=vector_database,
)
app = executor.run()

10:58:57 httpx INFO   HTTP Request: GET https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333 "HTTP/1.1 200 OK"
10:58:57 httpx INFO   HTTP Request: GET https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/file_monkesearch/exists "HTTP/1.1 200 OK"
10:58:57 httpx INFO   HTTP Request: GET https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/file_monkesearch "HTTP/1.1 200 OK"
10:58:57 httpx INFO   HTTP Request: PUT https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/file_monkesearch/index?wait=true "HTTP/1.1 200 OK"
10:58:57 httpx INFO   HTTP Request: PUT https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/file_monkesearch/index?wait=true "HTTP/1.1 200 OK"
10:58:57 httpx INFO   HTTP Request: PUT https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/file_monkes

In [7]:
from datetime import datetime
import pandas as pd
def date_to_unix(date_str):
    if date_str is None:
        return None
    return int(datetime.strptime(date_str, "%d/%m/%Y").timestamp())
# Natural language query parameters
params = {
    "natural_query": "Recent PDF files about fashion or sport",
    "limit": 5,
}

# Run the query with natural language interface
result = app.query(query, **params)

# Print the actual interpreted search parameters extracted by NLQ
print("Search Parameters Parsed by NLQ:")
print(result.metadata.search_params)

# Convert the results to a pandas DataFrame
df = sl.PandasConverter.to_pandas(result)

# Convert Unix timestamps to dd/mm/yyyy format in the DataFrame
import pandas as pd
df["Creation_Date"] = pd.to_datetime(df["Creation_Date"], unit="s").dt.strftime("%d/%m/%Y")
df["Last_Modified_Date"] = pd.to_datetime(df["Last_Modified_Date"], unit="s").dt.strftime("%d/%m/%Y")

display(df)

ValueError: Unknown query parameters: natural_query.

In [ ]:

# These are the names of the semantic/numeric spaces in the same order as in your index.spaces list
space_names = [
    "content_similarity",
    "filename_similarity",
    "size_score",
    "creation_date_score",
    "modified_date_score"
]

rows = []

for entry in result.entries:
    partial_scores = dict(zip(space_names, entry.metadata.partial_scores))
    row = {"id": entry.id, **partial_scores}
    rows.append(row)
    
df = pd.DataFrame(rows)
df


,id,content_similarity,filename_similarity,size_score,creation_date_score,modified_date_score
0,id_56,0.150118,0.168721,0.0,0.0,0.307981
1,id_68,0.140760,0.152928,0.0,0.0,0.276024
2,id_62,0.120002,0.159565,0.0,0.0,0.279851
3,id_89,0.095913,0.184917,0.0,0.0,0.275791
4,id_92,0.122882,0.159565,0.0,0.0,0.261333


In [ ]:
from datetime import datetime
import pandas as pd

def date_to_unix(date_str):
    if date_str is None:
        return None
    return int(datetime.strptime(date_str, "%d/%m/%Y").timestamp())

# Define your parameters with human-readable date strings or None
params = {
    # Text similarity inputs
    "content_query": "cinema fashion",
    "filename_query": "technical",

    # Weights for semantic spaces (matching what's in query.weights)
    "content_weight": 0.0,
    "filename_weight": 0.0,
    "size_weight": 0.0,
    "creation_date_weight": 1.0,
    "modified_date_weight": 1.0,

    # Weights for similarity matching
    "similar_content_weight": 0.0,
    "similar_filename_weight": 0.0,

    # Size filters (in kilobytes)
    #"min_size_kb": 1000,
    #"max_size_kb": 10000,

    # Date filters (input as "dd/mm/yyyy" strings or None)
    "min_creation_date": date_to_unix("03/08/2021"),
    "max_creation_date": date_to_unix("31/12/2025"),
    "min_modified_date": date_to_unix(None),  # No lower bound
    "max_modified_date": date_to_unix("30/06/2035"),

    # Categorical filters (must match query.py)
    "filetype_include_any": ['pdf', 'docx'],
    "tags_include_any": ["cinema", "fashion"],

    # Result limit
    "limit": 5,
}

# Run the query
result = app.query(query, **params)

# Inspect parameters used
print(result.metadata.search_params)

df = sl.PandasConverter.to_pandas(result)

if df.empty:
    print("No results found for the query.")
    # Optionally, return or display empty data
    display(df)  # or simply do nothing / return
else:
    # Convert Unix timestamps (in milliseconds or seconds) to formatted dates
    # Adjust unit='ms' or 's' as needed based on your data

    # Example using 'ms' if timestamps are in milliseconds
    df["Creation_Date"] = pd.to_datetime(df["Creation_Date"], unit="s").dt.strftime("%d/%m/%Y")
    df["Last_Modified_Date"] = pd.to_datetime(df["Last_Modified_Date"], unit="s").dt.strftime("%d/%m/%Y")

    display(df)



In [23]:
from datetime import datetime
import pandas as pd


def date_to_unix(date_str):
    if date_str is None:
        # Use zero for no lower bound, not None (filters expect int)
        return 0
    return int(datetime.strptime(date_str, "%d/%m/%Y").timestamp())


# Define your parameters matching the query.py
params = {
    # Semantic similarity text query per space
    "contenttype_query": "pdf document",
    "name_query": "CatastoAppartamento",

    # Weights for semantic spaces as defined in query weights
    "contenttype_weight": 1.0,
    "kind_weight": 1.0,
    "category_weight": 1.0,
    "name_weight": 0.0,
    "size_weight": 1.0,  # Filter only, no similarity weight on size
    "creation_date_weight": 0.0,
    "modified_date_weight": 0.0,

    # Size filters (bytes)
    "min_size_kb": 0,
    "max_size_kb": 10000,

    # Date filters (Unix timestamps in seconds)
    "min_creation_date": date_to_unix("03/08/2025"),
    "max_creation_date": date_to_unix("31/12/2025"),
    "min_modified_date": date_to_unix(None),  # 0 means no lower bound
    "max_modified_date": date_to_unix("30/06/2035"),

    # If you have categorical filters (make sure implemented in query)
    # "filetype_include_any": ['pdf', 'docx'],
    # "tags_include_any": ["cinema", "fashion"],

    # Limit of results
    "limit": 5,
}


# Run the query
result = app.query(query, **params)


# Inspect parameters used
print(result.metadata.search_params)


df = sl.PandasConverter.to_pandas(result)

if df.empty:
    print("No results found for the query.")
    display(df)
else:
    # Convert Unix timestamps (in seconds) to formatted date strings
    df["CreationDate"] = pd.to_datetime(df["CreationDate"], unit="s").dt.strftime("%d/%m/%Y")
    df["ContentChangeDate"] = pd.to_datetime(df["ContentChangeDate"], unit="s").dt.strftime("%d/%m/%Y")

    display(df)


16:35:40 superlinked.framework.query.query_dag_evaluator INFO   evaluated query


16:35:41 httpx INFO   HTTP Request: POST https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/file_monkesearch/points/query "HTTP/1.1 200 OK"
16:35:42 superlinked.framework.dsl.executor.query.query_executor INFO   executed query
{'contenttype_query': 'pdf document', 'similar_filter_TextSimilaritySpace_8831_FileDocument_ContentType_weight_param__': 1.0, 'kind_query': None, 'similar_filter_TextSimilaritySpace_ed01_FileDocument_Kind_weight_param__': 1.0, 'category_query': None, 'similar_filter_CategoricalSimilaritySpace_c7a6_FileDocument_ContentType_weight_param__': 1.0, 'name_query': 'CatastoAppartamento', 'similar_filter_TextSimilaritySpace_eba1_FileDocument_Name_weight_param__': 1.0, 'hard_filter_Size_be_greater_than_param__': 0.0, 'min_size_kb': 0.0, 'max_size_kb': 10000.0, 'min_creation_date': 1754172000, 'max_creation_date': 1767135600, 'min_modified_date': 0, 'max_modified_date': 2066767200, 'limit': 5, 'select_param__': ['ContentType', 'Kin

,ContentType,Kind,Name,Path,Size,CreationDate,ContentChangeDate,id,similarity_score
0,text/css,css,leaflet.awesome-markers.css,/home/biso/Downloads/Streamlit_files/leaflet.a...,2225.0,23/08/2025,23/08/2025,id_200,0.614059
1,unknown,unknown,dataset.jsonl,/home/biso/Downloads/dataset.jsonl,549.0,07/08/2025,07/08/2025,id_109,0.611418
2,text/javascript,javascript,leaflet.awesome-markers.js,/home/biso/Downloads/Streamlit_files/leaflet.a...,3789.0,23/08/2025,23/08/2025,id_184,0.571546
3,application/pgp-keys,pgp-keys,ssh-key-2025-08-30.key,/home/biso/Downloads/ssh-key-2025-08-30.key,1675.0,30/08/2025,30/08/2025,id_71,0.567931
4,application/vnd.exstream-package,vnd.exstream-package,ssh-key-2025-08-30.key.pub,/home/biso/Downloads/ssh-key-2025-08-30.key.pub,399.0,30/08/2025,30/08/2025,id_36,0.555531


In [40]:

# These are the names of the semantic/numeric spaces in the same order as in your index.spaces list
space_names = [
    "contenttype_score",
    "kind_space_score",
    "category_score",
    "name_score",
    "size_score",
    "creation_date_score",
    "modified_date_score",

]



rows = []

for entry in result.entries:
    partial_scores = dict(zip(space_names, entry.metadata.partial_scores))
    row = {"id": entry.id, **partial_scores}
    rows.append(row)
    
df = pd.DataFrame(rows)
df


,id,contenttype_score,kind_space_score,category_score,name_score,size_score,creation_date_score,modified_date_score
0,id_200,0.114105,0.0,0.0,0.0,0.499954,0.0,0.0
1,id_109,0.111464,0.0,0.0,0.0,0.499954,0.0,0.0
2,id_184,0.071592,0.0,0.0,0.0,0.499954,0.0,0.0
3,id_71,0.067977,0.0,0.0,0.0,0.499954,0.0,0.0
4,id_36,0.055577,0.0,0.0,0.0,0.499954,0.0,0.0


### Query with simple_query without LLM imported from monkeSearch leann-plus-temporal-search.py

In [52]:
import re
from datetime import datetime, timedelta
import pandas as pd
from superlinked import framework as sl

# --- TimeParser class reused ---
class TimeParser:
    def __init__(self):
        self.pattern = r'(?:(around|about|roughly|approximately)\s+)?(\d+)\s+(hour|day|week|month|year)s?(?:\s+ago)?'
        self.regex = re.compile(self.pattern, re.IGNORECASE)
        self.stop_words = {
            'in', 'at', 'of', 'by', 'as', 'me',
            'the', 'a', 'an', 'and', 'any',
            'find', 'search', 'list',
            'ago', 'back',
            'past', 'earlier',
        }

    def clean_text(self, text):
        words = text.split()
        cleaned = ' '.join(word for word in words if word.lower() not in self.stop_words)
        return cleaned

    def parse(self, text):
        cleaned_text = self.clean_text(text)
        matches = []
        for match in self.regex.finditer(cleaned_text):
            fuzzy = match.group(1)
            number = int(match.group(2))
            unit = match.group(3).lower()
            matches.append({
                'full_match': match.group(0),
                'fuzzy': bool(fuzzy),
                'number': number,
                'unit': unit,
                'range': self.calculate_range(number, unit, bool(fuzzy))
            })
        return matches

    def calculate_range(self, number, unit, is_fuzzy):
        units = {
            'hour': timedelta(hours=number),
            'day': timedelta(days=number),
            'week': timedelta(weeks=number),
            'month': timedelta(days=number * 30),
            'year': timedelta(days=number * 365)
        }
        delta = units[unit]
        now = datetime.now()
        target = now - delta
        if is_fuzzy:
            buffer = delta * 0.2
            start = (target - buffer).isoformat()
            end = (target + buffer).isoformat()
        else:
            start = target.isoformat()
            end = now.isoformat()
        return (start, end)


def date_to_unix(date_str):
    if date_str is None:
        return 0
    return int(datetime.strptime(date_str, "%d/%m/%Y").timestamp())



def search_files_superlinked(app, query, raw_query, top_k=15):
    parser = TimeParser()
    time_matches = parser.parse(raw_query)

    clean_query = raw_query
    if time_matches:
        for match in time_matches:
            clean_query = clean_query.replace(match['full_match'], '').strip()

    if len(clean_query) < 4:
        print("Error: add more input for accurate results.")
        return

    params = {
        "contenttype_query": clean_query,
        "kind_query": clean_query,
        "category_query": clean_query,
        "name_query": clean_query,

        "contenttype_weight": 1.0,
        "kind_weight": 1.0,
        "category_weight": 1.0,
        "name_weight": 0.0,
        "size_weight": 1.0,
        "creation_date_weight": 0.0,
        "modified_date_weight": 0.0,

        "min_size_kb": 0,
        "max_size_kb": 10000,

        "min_creation_date": 0,
        "max_creation_date": 4102444800,
        "min_modified_date": 0,
        "max_modified_date": 4102444800,

        "limit": top_k,
    }

    if time_matches:
        start_iso, end_iso = time_matches[0]['range']
        params["min_creation_date"] = int(datetime.fromisoformat(start_iso).timestamp())
        params["max_creation_date"] = int(datetime.fromisoformat(end_iso).timestamp())
        params["min_modified_date"] = int(datetime.fromisoformat(start_iso).timestamp())
        params["max_modified_date"] = int(datetime.fromisoformat(end_iso).timestamp())

    result = app.query(query, **params)

    print(f"\nSearch results for: '{raw_query}'")
    if time_matches:
        print(f"Time filter: {time_matches[0]['number']} {time_matches[0]['unit']}(s) {'(fuzzy)' if time_matches[0]['fuzzy'] else ''}")
        print(f"Date range: {start_iso[:10]} to {end_iso[:10]}")
    print("-" * 80)

    df = sl.PandasConverter.to_pandas(result)
    if df.empty:
        print("No results found.")
        return

    df["CreationDate"] = pd.to_datetime(df["CreationDate"], unit="s").dt.strftime("%d/%m/%Y")
    df["ContentChangeDate"] = pd.to_datetime(df["ContentChangeDate"], unit="s").dt.strftime("%d/%m/%Y")

    display(df)


# Example of manual execution in notebook:

# Be sure to initialize your Superlinked app and query beforehand.

raw_query = "image from approximately 1 month ago"
top_k = 10
search_files_superlinked(app, query, raw_query, top_k)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

17:24:29 superlinked.framework.query.query_dag_evaluator INFO   evaluated query
17:24:31 httpx INFO   HTTP Request: POST https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/file_monkesearch/points/query "HTTP/1.1 200 OK"
17:24:31 superlinked.framework.dsl.executor.query.query_executor INFO   executed query

Search results for: 'image from approximately 1 month ago'
Time filter: 1 month(s) (fuzzy)
Date range: 2025-08-30 to 2025-09-11
--------------------------------------------------------------------------------


,ContentType,Kind,Name,Path,Size,CreationDate,ContentChangeDate,id,similarity_score
0,application/json,json,veneto_poi.json,/home/biso/Downloads/veneto_poi.json,1393.0,02/09/2025,02/09/2025,id_121,0.561631
1,application/json,json,veneto_locations (1).json,/home/biso/Downloads/veneto_locations (1).json,1393.0,02/09/2025,02/09/2025,id_32,0.561631
2,application/json,json,veneto_locations.json,/home/biso/Downloads/veneto_locations.json,1601.0,02/09/2025,02/09/2025,id_82,0.561631
